In [1]:
import cv2 as cv
import numpy as np
import math
import random
from sklearn.cluster import KMeans
from itertools import chain

In [2]:
# Regulation Table Tennis dimensions in cm
TBL_WIDTH = 152.5
TBL_LENGTH = 274
TBL_ASPECT_RATIO = TBL_WIDTH/TBL_LENGTH
# For these two pairs refer to the following lines 0: Bottom Right, 1: Top Left, 2: Top Right, 3: Bottom Left
LINE_PAIRS = [[0, 3], [3, 1], [1, 2], [2, 0]] # Botttom, Left, Top, Right
TABLE_COORDS = np.array([[TBL_WIDTH, TBL_LENGTH], [0, TBL_LENGTH], [0, 0], [TBL_WIDTH, 0]], np.float32)

In [3]:
min_table_size = 8_000
img_sz = (960, 540)

In [4]:
def find_outliers(data, m = 5):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    outliers = np.array([False if x < m else True for x in s])
    return outliers

Equation of the tangent line at $(x_0, y_0)$ to the circle with radius $\rho$ and center at the origin is: $xx_0+yy_0=\rho^2$ where $(x_0, y_0)=(\rho\cos\theta, \rho\sin\theta)$\
This gives the equation: $x\cos\theta+y\sin\theta=\rho$\
In cartesian form: 

In [5]:
def find_intersections(lines):
    # Sort lines by theta so the lines are split into pairs of parallel lines
    lines = lines[lines[:, 1].argsort()]
    pts = []
    for i in range(len(LINE_PAIRS)):
        line_0, line_1 = lines[LINE_PAIRS[i]]
        rho_0, theta_0 = line_0
        rho_1, theta_1 = line_1
        b_0 = rho_0/math.sin(theta_0)
        b_1 = rho_1/math.sin(theta_1)
        m_0 = -math.tan(theta_0)**-1
        m_1 = -math.tan(theta_1)**-1
        x = (b_1-b_0)/(m_0-m_1)
        y = (b_1*m_0 - b_0*m_1)/(m_0-m_1)
        pts.append([x, y])
    pts = np.array(pts, np.float32)
    return pts

In [6]:
def handleMouseClick(event, x, y, flags, param):
    # if event == cv.EVENT_LBUTTONUP:
    t_pts = cv.transform(np.array([[x, y]], np.float32).reshape((-1, 1, 2)), param["t"])
    t_pts = t_pts.reshape((1, 3))
    t_pts = t_pts[:,:2] / t_pts[:, -1].reshape(-1, 1) # Normalize using third column
    t_pts = t_pts.astype(np.int32)[0] # Convert from float to int, and flit to match Numpy convention
    frame = np.zeros((274, 153)) # y, x
    frame[137] = np.ones((1, 153))*255
    frame[:, 77] = np.ones((274, 1)).ravel()*255
    frame = cv.circle(frame, t_pts, 5, (255, 255, 255), -1)
    cv.imshow("Location", frame)

In [7]:
# Use Hough Lines and the table dimensions to determine the corner positions
def find_corners(lines, areas, frame):
    outliers = find_outliers(areas)
    lines = lines[np.invert(outliers)]
    lines = np.array(list(chain.from_iterable(lines)))
    kmeans = KMeans(n_clusters=4, random_state=0).fit(lines)
    pts = find_intersections(kmeans.cluster_centers_)
    tform = cv.getPerspectiveTransform(pts, TABLE_COORDS, cv.DECOMP_LU)
    # t_pts = cv.transform(pts.reshape((-1, 1, 2)), tform)
    # t_pts = t_pts.reshape((4, 3))
    # t_pts = t_pts[:,:2] / t_pts[:, -1].reshape(-1, 1) # Normalize using third column
    corners = pts.astype(np.int32)
    corners = np.reshape(corners, (-1,1,2))
    overlay = frame.copy()
    cv.fillPoly(overlay,[corners],(0,255,255))
    overlay = cv.addWeighted(overlay, 0.5, frame, 0.5, 0)
    cv.imshow("frame", overlay)
    cv.setMouseCallback('frame', lambda event, x, y, flags, param: handleMouseClick(event, x, y, flags, param={"t": tform}))
    cv.waitKey(0)
    cv.destroyAllWindows()
    cv.waitKey(1)

In [8]:
def image_segmentation(input_video_path):
    table_color_lower = (107, 0, 0)
    table_color_upper = (120, 255, 255)
    capture = cv.VideoCapture(input_video_path)
    num_frames = int(capture.get(cv.CAP_PROP_FRAME_COUNT))
    resize_factor = 10
    n_samples = 7
    table_areas = []
    table_lines = []
    # Randomly sample n_samples frames from the video to get the best chance of finding frames of unobstructed table
    # Save the contour area values to identify outliers, only use nonn-outlier values to find table lines
    random_frames = random.sample(range(num_frames), n_samples)
    for i in random_frames:
        capture.set(cv.CAP_PROP_POS_FRAMES, i)
        ret, frame = capture.read()
        if frame is None:
            break
        # Normalize image size
        frame = cv.resize(frame, img_sz)
        blurred = cv.GaussianBlur(frame, (3, 3), 0)
        hsv = cv.cvtColor(blurred, cv.COLOR_BGR2HSV)
        mask = cv.inRange(hsv, table_color_lower, table_color_upper)
        kernel = np.ones((4, 4), np.uint8)
        mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
        mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
        contours, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        contours = list(contours)
        # filter out contours that are too small
        # filter out contours that are too big
        # find the contour with the closest center to the center of the frame
        table_cnt = None
        min_center_dist = (img_sz[0]**2 + img_sz[1]**2)**0.5 # initialize to largest possible value (corner of frame,furthest from center)
        frame_center = tuple([dim/2 for dim in img_sz])
        for cnt in contours:
            if cv.contourArea(cnt) > min_table_size:
                moments = cv.moments(cnt)
                cX = int(moments["m10"] / moments["m00"])
                cY = int(moments["m01"] / moments["m00"])
                euclid_dst = ((frame_center[0] - cX)**2 + (frame_center[1] - cY)**2)**0.5
                if euclid_dst < min_center_dist:
                    min_center_dist = euclid_dst
                    table_cnt = cnt
        # Use thin contour border crucial to increasing accuracy of the angle of the lines created
        cnt_img = cv.drawContours(np.zeros((mask.shape[0],mask.shape[1]), np.uint8), [table_cnt], 0, (255, 255, 255), 1)
        # rho: specifies how straight the line has to be, lower => stricter
        # theta: specifies how similar the lines need to be to be considered the same line, higher => combines more lines
        # threshold: specifies how many points leed to lie on the line to be considered a line, higher => stricter
        lines = cv.HoughLines(cnt_img, 1, 2*np.pi / 180, 50, None, 0, 0)
        lines = lines.reshape((-1, 2))
        table_areas.append(cv.contourArea(table_cnt))
        table_lines.append(lines)
        keyboard = cv.waitKey(30)
        if keyboard == 'q' or keyboard == 27:
            break
    table_lines = np.array(table_lines)
    find_corners(table_lines, table_areas, frame)
    capture.release()

In [9]:
for video_num in range(0, 10):
    input_video_path = f"./imp/videos/videos_0/{video_num}.mov"
    image_segmentation(input_video_path)
cv.destroyAllWindows()
cv.waitKey(1)

/var/folders/s3/v3l8xgqd3xdgc1ll_pmdv1qr0000gn/T/ipykernel_87374/855033017.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  table_lines = np.array(table_lines)
2022-12-18 18:22:05.380 Python[87374:2466984] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/s3/v3l8xgqd3xdgc1ll_pmdv1qr0000gn/T/org.python.python.savedState
